In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [21]:
def load_and_clean_csv(file_path):
    df = pd.read_csv(file_path)

    required_columns = [
        "product_category",
        "product_id",
        "transaction_qty",
        "transaction_date"
    ]

    df = df[required_columns].copy()

    df["transaction_date"] = pd.to_datetime(
    df["transaction_date"],
    format="mixed",
    dayfirst=True,
    errors="coerce"
)


    # optional: drop rows where date could not be parsed
    df = df.dropna(subset=["transaction_date"])

    return df


In [4]:
def aggregate_monthly_sales(df):
    df["year"] = df["transaction_date"].dt.year
    df["month"] = df["transaction_date"].dt.month

    monthly_sales = (
        df.groupby(["product_id", "product_category", "year", "month"])
          ["transaction_qty"]
          .sum()
          .reset_index()
          .sort_values(["product_id", "year", "month"])
    )

    return monthly_sales


In [5]:
def create_time_features(df):
    df["lag_1"] = df.groupby("product_id")["transaction_qty"].shift(1)
    df["lag_3_avg"] = (
        df.groupby("product_id")["transaction_qty"]
          .shift(1)
          .rolling(3)
          .mean()
    )

    df = df.dropna().reset_index(drop=True)
    return df


In [6]:
def apply_month_weight(df, target_month):
    df["month_weight"] = np.where(
        df["month"] == target_month,
        0.8,   # same month last year
        0.2    # other months
    )
    return df


In [7]:
def prepare_training_data(df):
    X = df[
        ["product_id", "month", "year", "lag_1", "lag_3_avg", "month_weight"]
    ]
    y = df["transaction_qty"]

    return X, y


In [8]:

def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=False
    )

    model = RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    rmse = mean_squared_error(y_test, preds) ** 0.5
    r2 = r2_score(y_test, preds)

    print("📊 Model Accuracy Metrics")
    print(f"MAE  : {mae:.2f}")
    print(f"RMSE : {rmse:.2f}")
    print(f"R²   : {r2:.2f}")

    return model


In [9]:
def predict_next_month(model, df, target_year, target_month):
    latest = (
        df.sort_values(["product_id", "year", "month"])
          .groupby("product_id")
          .tail(1)
          .copy()
    )

    latest["previous_month_sales"] = latest["lag_1"]

    latest["month"] = target_month
    latest["year"] = target_year

    features = latest[
        ["product_id", "month", "year", "lag_1", "lag_3_avg", "month_weight"]
    ]

    latest["predicted_sales"] = model.predict(features)

    return latest


In [14]:
def generate_stock_report(df, threshold_multiplier=1.2):
    avg_sales = df["predicted_sales"].mean()
    threshold = avg_sales * threshold_multiplier

    df["stock_status"] = np.where(
        df["predicted_sales"] >= threshold,
        "HIGH STOCK REQUIRED",
        "NORMAL STOCK"
    )

    # % change calculation (safe)
    df["stock_change_pct"] = np.where(
        df["previous_month_sales"] > 0,
        ((df["predicted_sales"] - df["previous_month_sales"])
         / df["previous_month_sales"]) * 100,
        np.nan
    )

    # round for readability
    df["stock_change_pct"] = df["stock_change_pct"].round(2)

    return df[
        [
            "product_id",
            "product_category",
            "previous_month_sales",
            "predicted_sales",
            "stock_change_pct",
            "stock_status"
        ]
    ]


In [23]:
# Load user CSV
df = load_and_clean_csv("Coffee Shop Sales.csv")


# Monthly aggregation
monthly_sales = aggregate_monthly_sales(df)

# Feature engineering
monthly_sales = create_time_features(monthly_sales)

# Example: Predict JULY (7)
monthly_sales = apply_month_weight(monthly_sales, target_month=7)

# Prepare data
X, y = prepare_training_data(monthly_sales)

# Train model
model = train_model(X, y)

# Predict future month
predictions = predict_next_month(
    model,
    monthly_sales,
    target_year=2024,
    target_month=6
)

# Generate stock report
stock_report = generate_stock_report(predictions)

print(stock_report)


📊 Model Accuracy Metrics
MAE  : 47.23
RMSE : 60.37
R²   : 0.84
     product_id product_category  previous_month_sales  predicted_sales  \
8             1     Coffee beans                  13.0        21.290917   
17            2     Coffee beans                   5.0        19.400000   
24            3     Coffee beans                   6.0        19.235000   
33            4     Coffee beans                   9.0        18.822500   
40            5     Coffee beans                  10.0        18.692500   
..          ...              ...                   ...              ...   
669          81          Branded                  13.0        95.260509   
677          82          Branded                  14.0       103.811134   
685          83          Branded                  20.0       100.160739   
693          84         Flavours                 171.0       281.428746   
701          87           Coffee                  17.0       109.705093   

     stock_change_pct         stock_

In [ ]:
def load_and_clean_csv(file_path):
    df = pd.read_csv(file_path)

    required_columns = [
        "product_category",
        "product_id",
        "transaction_qty",
        "transaction_date"
    ]

    df = df[required_columns].copy()

    df["transaction_date"] = pd.to_datetime(
    df["transaction_date"],
    format="mixed",
    dayfirst=True,
    errors="coerce"
)


    # optional: drop rows where date could not be parsed
    df = df.dropna(subset=["transaction_date"])

    return df
